# Chapter 2.6. Intro to TensorFlow and Keras

Original code in Nikolenko book was made in **TensorFlow 1.x (TF1)**, but here I will convert examples to **TensorFlow 2.0 (TF2)**

In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf

## 1. Simple Regression using TensorFlow 2

$$y = k_{true} * x + b_{true} + \epsilon$$

$$\epsilon - normal noise (N(0, 2))$$

Let's try to recover coefs `k_true`, `b_true`

In [2]:
# set constants
n_samples = 1000
batch_size = 100
num_steps = 2000
display_step = 100

k_true = 2
b_true = 1

# generate data
X_data = np.random.uniform(1,10,(n_samples, 1))
y_data = k_true * X_data + b_true + np.random.normal(0, 2, (n_samples, 1))

# initialize coefs
k = tf.Variable(tf.random.normal((1,1), dtype='float64'), name='slope')
b = tf.Variable(tf.zeros((1,), dtype='float64'), name='bias')


# create model class
class MyModel():
    
    def __init__(self, k, b):
        self.k = k
        self.b = b
        
    def __call__(self, X):
        return tf.matmul(X, self.k) + self.b
    
# define loss
def mse_loss(y_true, y_pred):
    return tf.reduce_sum(tf.square(y_true - y_pred))

# define forward step
def train(model, X, y, optimizer):
    """
    One step of optimization
    
    params:
        model: model object ot train
        X: data
        y: target 
    return:
        loss: loss value
    """
    with tf.GradientTape() as t:
        y_pred = model(X)
        loss = mse_loss(y, y_pred)
        
    grads = t.gradient(loss, [model.k, model.b])
    optimizer.apply_gradients(zip(grads,[model.k, model.b]))
    
    return loss

In [3]:
optimizer = tf.optimizers.SGD(learning_rate=1e-4)
model = MyModel(k, b)

# running optimization
for i in range(num_steps):
    
    # select data batch
    indices = np.random.choice(n_samples, batch_size)
    X_batch, y_batch = X_data[indices], y_data[indices]
    
    loss_val = train(model, X_batch, y_batch, optimizer)
    
    # output info
    if (i+1) % display_step == 0:
        print(f'Epoch {i+1}: loss={loss_val}, k={model.k.numpy()}, b={model.b.numpy()}')

Epoch 100: loss=328.4014782744676, k=[[2.0347777]], b=[0.45696314]
Epoch 200: loss=441.4016353059742, k=[[2.03287446]], b=[0.61575231]
Epoch 300: loss=428.30945380147494, k=[[2.07178783]], b=[0.74163124]
Epoch 400: loss=424.43686321990606, k=[[2.04131705]], b=[0.82955066]
Epoch 500: loss=335.6280162414797, k=[[1.99352792]], b=[0.87748055]
Epoch 600: loss=371.01780966592855, k=[[1.98934505]], b=[0.95281984]
Epoch 700: loss=412.8920851745517, k=[[2.02760397]], b=[0.99698293]
Epoch 800: loss=408.2759653164994, k=[[2.06301567]], b=[1.00223127]
Epoch 900: loss=432.59193086789526, k=[[1.97749149]], b=[1.01195023]
Epoch 1000: loss=516.9335502605454, k=[[2.00373984]], b=[1.01602768]
Epoch 1100: loss=381.3437039336857, k=[[2.03720823]], b=[1.00947445]
Epoch 1200: loss=375.30038921320687, k=[[1.96749361]], b=[1.00676618]
Epoch 1300: loss=386.22566716969754, k=[[2.01734126]], b=[1.02658512]
Epoch 1400: loss=446.9548052102519, k=[[2.02009147]], b=[1.04640993]
Epoch 1500: loss=332.3382147063178, k=

We got approximate values `k = 1.97` and `b = 1.03`.


## 2. Logistic Regression using TF2 Keras api

- Generate some data;
- Define simple logistic regression (1 dense layer with sigmoid activation);
- Train model

In [4]:
# generate data
def sampler(n, x, y):
    return np.random.normal(size=[n,2]) + [x, y]

def sample_data(n=1000, p0=(-1., -1.), p1=(1., 1.)):
    zeros, ones = np.zeros((n, 1)), np.ones((n, 1))
    labels = np.vstack([zeros, ones])
    
    z_sample = sampler(n, x=p0[0], y=p0[1])
    o_sample = sampler(n, x=p1[0], y=p1[1])
    
    return np.vstack([z_sample, o_sample]), labels

X_train, Y_train = sample_data()
X_test, Y_test = sample_data(100)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(2000, 2) (2000, 1)
(200, 2) (200, 1)


In [5]:
# define model
model = tf.keras.Sequential(
    tf.keras.layers.Dense(1, activation='sigmoid')
)
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [6]:
# train model
model.fit(X_train, Y_train, epochs=50, batch_size=16, validation_data=(X_test, Y_test))

Train on 2000 samples, validate on 200 samples
Epoch 1/50
2000/2000 [==============================] - 0s 204us/sample - loss: 0.2649 - accuracy: 0.9180 - val_loss: 0.2683 - val_accuracy: 0.9000
Epoch 2/50
2000/2000 [==============================] - 0s 45us/sample - loss: 0.2479 - accuracy: 0.9205 - val_loss: 0.2555 - val_accuracy: 0.9000
Epoch 3/50
2000/2000 [==============================] - 0s 42us/sample - loss: 0.2365 - accuracy: 0.9225 - val_loss: 0.2470 - val_accuracy: 0.9050
Epoch 4/50
2000/2000 [==============================] - 0s 43us/sample - loss: 0.2285 - accuracy: 0.9220 - val_loss: 0.2412 - val_accuracy: 0.9050
Epoch 5/50
2000/2000 [==============================] - 0s 43us/sample - loss: 0.2224 - accuracy: 0.9215 - val_loss: 0.2369 - val_accuracy: 0.9100
Epoch 6/50
2000/2000 [==============================] - 0s 43us/sample - loss: 0.2178 - accuracy: 0.9215 - val_loss: 0.2337 - val_accuracy: 0.9100
Epoch 7/50
2000/2000 [==============================] - 0s 43us/sample

We got `train accuracy = 0.9230` and `validation accuracy = 0.9050`